<a href="https://colab.research.google.com/github/SeongDeokKo/GuKellyXiu_Replicate/blob/main/Estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd 
from collections import Counter
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys
sys.path.append('/content/drive/MyDrive/')
import numpy as np
import pandas as pd 
from collections import Counter
import FunLib_stock as FL
import multiprocessing as mp

In [3]:
new_firm_data = pd.read_csv('/content/drive/MyDrive/x_y_wo_inter.csv')

In [4]:
new_firm_data[new_firm_data.columns[2:96]] = new_firm_data[new_firm_data.columns[2:96]].astype('float32')
new_firm_data[new_firm_data.columns[0:2]] = new_firm_data[new_firm_data.columns[0:2]].astype('int32')
new_firm_data[new_firm_data.columns[96:170]] = new_firm_data[new_firm_data.columns[96:170]].astype('int8')
new_firm_data[new_firm_data.columns[170]] = new_firm_data[new_firm_data.columns[170]].astype('float32')

In [5]:

def R2OOS(y_true, y_forecast):
    
    import numpy as np
   
    SSres = np.nansum(np.square(y_true-y_forecast))
    SStot = np.nansum(np.square(y_true))

    return 1-SSres/SStot



# =========================================================================
#   PCR, 94 + dummy variable(no intersection term), Use cross-validation to select the number of PCA components  
# =========================================================================

def Pca_regression(X,Y,numpc,num_t_v):
    # numpc (list) : # of principal component ex[3,4,5,6,7]
    # num_t_v (list) : # of training set & cross-val set   ex[100, 10]
    # X consists of Traing, Val and Test set
    
    import numpy as np 
    from sklearn.decomposition import PCA
    from sklearn.linear_model import LinearRegression
    from sklearn.preprocessing import StandardScaler
    from sklearn.metrics import mean_squared_error
    
    num_train = num_t_v[0]
    num_val = num_t_v[1]
    num_test = X.shape[0] - (num_train + num_val)
    
    # Split data into training and test
    X_train = X[:num_train,:]
    Y_train = Y[:num_train,:]
    
    X_val = X[num_train:(num_train+num_val),:]
    Y_val = Y[num_train:(num_train+num_val),:]
    
    X_test = X[(num_train+num_val):,:]
    
       
    # Scale Inputs for Training
    X_scaler = StandardScaler()
    X_train_scaled = X_scaler.fit_transform(X_train)
    
    X_val_scaled = X_scaler.transform(X_val)
    X_test_scaled = X_scaler.transform(X_test)
    
    
    # use cross-validation mean-squared-error to determine the number of component 
    mse = np.full((len(numpc),1),np.nan, dtype = np.float32)

    for i in range(len(numpc)):
        pca = PCA(n_components = numpc[i])
        principalComponents = pca.fit_transform(X_train_scaled)
        
        X_val_weighted = pca.transform(X_val_scaled)
        
        line_fitter = LinearRegression()
        line_fitter.fit(principalComponents, Y_train)
        
        Ypred_val = np.full((num_val,1),np.nan, dtype = np.float32)
        for j in range(num_val):
            Ypred_val[j,0] = line_fitter.predict(X_val_weighted[j,:].reshape(1,-1))
                   
        mse[i,0] = mean_squared_error(Y_val.reshape(-1), Ypred_val.reshape(-1))
    
    
    argmin_numpc = numpc[np.argmin(mse)]
    
    pca = PCA(n_components = argmin_numpc)
    principalComponents = pca.fit_transform(X_train_scaled)
    
    X_test_weighted = pca.transform(X_test_scaled)
    
    line_fitter = LinearRegression()
    line_fitter.fit(principalComponents, Y_train)
        
    Ypred_test = np.full((num_test,1),np.nan, dtype = np.float32)
    for j in range(num_test):
        Ypred_test[j,0]=line_fitter.predict(X_test_weighted[j,:].reshape(1,-1))
        
          
    return Ypred_test, argmin_numpc




# =========================================================================
#   PLS, 94 + dummy variable(no intersection term), Use cross-validation to select the number of components  
# =========================================================================





In [102]:
def elastic_net(X,Y,num):
    # num_t_v (list) : # of training set & cross-val set   ex[100, 10]
    # X consists of Traing, Val and Test set
    
    import numpy as np
    from sklearn.linear_model import ElasticNetCV
    from sklearn.preprocessing import StandardScaler
    from sklearn.model_selection import PredefinedSplit
    
    num_test = sum(num[420:])
    num_start = sum(num[0:84])
    Ypred_test = np.full((num_test,1),np.nan, dtype = np.float32)
    for i in range(25):
      num_trian = sum(num[84:(276 + 12*i)])
      num_val = sum(num[(276 + 12*i):(420 + 12*i)])
      num_test_start = sum(num[420:]) - sum(num[420+12*i:])
      num_pred = sum(num[420+12*i:420+12*(i+1)])
      if i == 24:
        num_pred = sum(num[(420+12*i):])
      # Split data into training and test
      X_train = X[num_start:(num_start + num_train+num_val),:]   # train + validation
      Y_train = Y[num_start:(num_start + num_train+num_val),:]   # train + validation
    
      # Scale Inputs for Training
      X_scaler = StandardScaler()
      X_train_scaled = X_scaler.fit_transform(X_train)
      XX = X_scaler.transform(X)
    
      # pre-define validation 
      test_fold =  np.concatenate(((np.full((num_train),-1),np.full((num_val),0))))
      ps = PredefinedSplit(test_fold.tolist())
    
      # fit & predict 
      model = ElasticNetCV(cv=ps, max_iter=5000, n_jobs=-1, l1_ratio=[.1, .3, .5, .7, .9], 
                         random_state=42)
      model = model.fit(X_train_scaled, Y_train.reshape(-1))
    
      for j in range(num_pred):
        Ypred_test[num_test_start + j,0]=model.predict(XX[ num_start + num_train + num_val +j,:].reshape(1,-1))
        
    return Ypred_test





In [127]:
def Pls_regression(X,Y,numpls,num):
    # numpls (list) : # of component ex[3,4,5,6,7]
    # num_t_v (list) : # of training set & cross-val set   ex[100, 10]
    # X consists of Traing, Val and Test set
    
    import numpy as np 
    from sklearn.cross_decomposition import PLSRegression
    from sklearn.preprocessing import StandardScaler
    from sklearn.metrics import mean_squared_error
    
    num_test = sum(num[420:])
    num_start = sum(num[0:84])
    Ypred_test = np.full((num_test,1),np.nan, dtype = np.float32)
    for i in range(25):
      num_trian = sum(num[84:(276 + 12*i)])
      num_val = sum(num[(276 + 12*i):(420 + 12*i)])
      num_test_start = sum(num[420:]) - sum(num[420+12*i:])
      num_pred = sum(num[420+12*i:420+12*(i+1)])
      if i == 24:
        num_pred = sum(num[(420+12*i):])
      # Split data into training and test
      X_train = X[num_start:(num_start +num_train),:]   # train
      Y_train = Y[num_start:(num_start +num_train),:]   # train 
      X_val = X[num_start:(num_start+num_train+num_val),:]
      Y_val = Y[num_start:(num_start+num_train+num_val),:]
      # Scale Inputs for Training
      X_scaler = StandardScaler()
      X_train_scaled = X_scaler.fit_transform(X_train)
      X_val_scaled = X_scaler.transform(X_val)
      XX = X_scaler.transform(X)

      # use cross-validation mean-squared-error to determine the number of component 
      mse = np.full((len(numpls),1),np.nan, dtype = np.float32)
      
      for j in range(len(numpls)):
          pls = PLSRegression(n_components = numpls[j])
          pls.fit(X_train_scaled, Y_train)
                
          Ypred_val = np.full((num_val,1),np.nan, dtype = np.float32)
          for k in range(num_val):
              Ypred_val[j,0]=pls.predict(X_val_scaled[k,:].reshape(1,-1))          
          mse[j,0] = mean_squared_error(Y_val.reshape(-1), Ypred_val.reshape(-1))
    
      argmin_numpls = numpls[np.argmin(mse)]
      pls = PLSRegression(n_components = argmin_numpls)
      pls.fit(X_train_scaled, Y_train)
    
      for u in range(num_pred):
        Ypred_test[num_test_start+u,0]=pls.predict(XX[ num_start + num_train + num_val + u,:].reshape(1,-1))          
              
    return Ypred_test


In [121]:
X = X_no_inter.to_numpy()
Y = y.to_numpy().reshape(-1,1)

In [115]:
num_test = sum(num[420:])
num_start = sum(num[0:84])
Ypred_test = np.full((num_test,1),np.nan, dtype = np.float32)
i = 1
num_trian = sum(num[84:(276 + 12*i)])
num_val = sum(num[(276 + 12*i):(420 + 12*i)])
num_test_start = sum(num[420:]) - sum(num[420+12*i:])
num_pred = sum(num[420+12*i:420+12*(i+1)])
if i == 24:
  num_pred = sum(num[(420+12*i):])
# Split data into training and test
      

In [123]:
from sklearn.preprocessing import StandardScaler

In [124]:
X_train = X[num_start:(num_start + num_train),:]   # train
Y_train = Y[num_start:(num_start +num_train),:]   # train 
X_val = X[num_start:(num_start+num_train+num_val),:]
Y_val = Y[num_start:(num_start+num_train+num_val),:]
# Scale Inputs for Training
X_scaler = StandardScaler()
X_train_scaled = X_scaler.fit_transform(X_train)
X_val_scaled = X_scaler.transform(X_val)
XX = X_scaler.transform(X)

In [126]:
X_val_scaled

array([[ 0.09353273, -0.54263586, -0.6080939 , ..., -0.05478966,
        -0.15216303,  0.        ],
       [-0.3703877 , -0.69055575, -0.69403875, ..., -0.05478966,
        -0.15216303,  0.        ],
       [-0.16017693, -0.97590506, -0.8335567 , ..., -0.05478966,
        -0.15216303,  0.        ],
       ...,
       [ 1.5197862 , -1.1758326 , -0.9106885 , ..., -0.05478966,
        -0.15216303,  0.        ],
       [-0.3282353 , -0.24680927, -0.408312  , ..., -0.05478966,
        -0.15216303,  0.        ],
       [-0.36953044, -1.7310886 , -1.0357853 , ..., -0.05478966,
        -0.15216303,  0.        ]], dtype=float32)

In [8]:
# X_no_inter (168) : 195703 ~ 201612, y(1) :195704 ~ 201701
# X-195703 & y-195704 are in the same row. 

X_no_inter = new_firm_data.iloc[:,2:170]     # without intersect terms
y = new_firm_data.iloc[:,-1]

In [13]:
sum(num[0:72])

85025

In [39]:

num_est = 1  # We estimate parameter 1 time, here not use

# if we estimate parameters more than 1 time(i.e using longer data), we should set below # recursively
# Train 
# Validation 
# Test 

num_start = sum(num[0:84])
num_train = sum(num[84:(216+12*5)])
num_val = sum(num[(216+12*5):(216+144+60)])
num_test = sum(num[(216+144+60):])
num_t_v = [num_train, num_val]

In [43]:
num_start, num_train , num_val , num_test

(109501, 662229, 884336, 2053843)

In [23]:
y_true = y.iloc[-num_test:].to_numpy().reshape(-1,1)  # for caluclating R2oos

In [ ]:

# Computational Ressources: Determine Number of available cores
ncpus = mp.cpu_count()
print("CPU count is: "+str(ncpus))



CPU count is: 4


In [80]:
sum(num[420+12*22:420+12*23]), sum(num[420+12*23:]) , sum(num[420+12*24:])

(67665, 125154, 56442)

In [85]:
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import LinearRegression

In [84]:
Y_pred_ols = np.full((num_test,1),np.nan, dtype = np.float32 )
Y_pred_ols_huber = np.full((num_test,1),np.nan, dtype = np.float32 )

Rolling Version 

i = 0, 64-80, 80-92, 92~93 ,

i = 1, 64-81, 81-93, 93~94

;;;;
i = 24, 64-04, 04-16, 16-16년 12월까지 

In [94]:
Y_pred_ols = np.full((num_test,1),np.nan, dtype = np.float32 )
for i in range(25):
  num_trian = sum(num[84:(276 + 12*i)])
  num_val = sum(num[(276 + 12*i):(420 + 12*i)])
  num_test_start = sum(num[420:]) - sum(num[420+12*i:])
  num_pred = sum(num[420+12*i:420+12*(i+1)])
  X_train_ols = X_no_inter.iloc[num_start:(num_start+ num_train+num_val),:].to_numpy()
  y_train_ols = y.iloc[num_start:(num_start +num_train+num_val)].to_numpy()
  reg = LinearRegression().fit(X_train_ols, y_train_ols)
  reg_huber = HuberRegressor(max_iter=500, alpha=0).fit(X_train_ols, y_train_ols)
  if i == 24:
    print('it is 24')
    num_pred = sum(num[(420+12*i):])
  for j in range(num_pred):
    Y_pred_ols[num_test_start + j,0] = reg.predict(X_no_inter.iloc[(num_start + num_train + num_val + j),:].to_numpy().reshape(1,-1)) 
    Y_pred_ols_huber[num_test_start + j,0] = reg_huber.predict(X_no_inter.iloc[(num_start + num_train+num_val+j),:].to_numpy().reshape(1,-1) )

it is 24


In [96]:
print('R2OOS, Huber Loss - Linear regression without intersection terms : ', FL.R2OOS(y_true, Y_pred_ols))

R2OOS, Huber Loss - Linear regression without intersection terms :  -0.0017085075378417969


In [95]:
print('R2OOS, Huber Loss - Linear regression without intersection terms : ', FL.R2OOS(y_true, Y_pred_ols_huber))

R2OOS, Huber Loss - Linear regression without intersection terms :  0.0004981160163879395


In [ ]:
Y_pred_elastic = elastic_net(X_no_inter.to_numpy(), y.to_numpy().reshape(-1,1), num)
numpls = [3,5,10,15,20]


In [109]:
print('R2OOS, Elastic-net - without intersection terms : ', FL.R2OOS(y_true, Y_pred_elastic))

R2OOS, Elastic-net - without intersection terms :  0.0016314983367919922


In [ ]:
Y_pred_pls = Pls_regression(X_no_inter.to_numpy(), y.to_numpy().reshape(-1,1) ,numpls,num)

In [ ]:
print('R2OOS, Huber Loss - Linear regression without intersection terms : ', FL.R2OOS(y_true, Y_pred_ols_huber))
print('R2OOS, Principal Components Regression - without intersection terms : ', FL.R2OOS(y_true, Y_pred_pca)) 
print('R2OOS, Partial Least Square - without intersection terms : ', FL.R2OOS(y_true, Y_pred_pls)) 
print('R2OOS, Elastic-net - without intersection terms : ', FL.R2OOS(y_true, Y_pred_elastic))
print('R2OOS, generalized linear - without intersection terms / with knots : ', FL.R2OOS(y_true, Y_pred_general_lin))

R2OOS, Huber Loss - Linear regression without intersection terms :  -0.0006206035614013672
R2OOS, Principal Components Regression - without intersection terms :  -0.015273094177246094
R2OOS, Partial Least Square - without intersection terms :  -1.157390536072052
R2OOS, Elastic-net - without intersection terms :  0.0024675253993113877
R2OOS, generalized linear - without intersection terms / with knots :  0.001908131940232649
